# Neptune Result Fetching Notebook

In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files

debug_print: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-09-23"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

import neptune # for logging progress and results
from neptune.types import File
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

## Gets the notebook filepath for Neptune:
import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
# notebook_filepath

neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortUpdated",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
        
neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])
# neptuner
neptuner.project

Automatic pdb calling has been turned OFF
TODAY_DAY_DATE: 2024-09-23_Apogee
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/


In [2]:
# neptuner.project.get_structure()
# neptuner.project.get_root_object() # neptune.metadata_containers.project.Project

# models_table_df = neptuner.project.fetch_models_table().to_pandas()
# models_table_df

_neptuner_runs_table = neptuner.project.fetch_runs_table()
runs_table_df: pd.DataFrame = _neptuner_runs_table.to_pandas()
runs_table_df

Fetching table...: 0 [00:00, ?/s]

,sys/creation_time,sys/description,sys/failed,sys/group_tags,sys/hostname,sys/id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,sys/state,sys/tags,sys/trashed,animal,exper_name,format_name,session_descriptor_string,session_name,source_code/entrypoint,source_code/git,monitoring/0003ad6d/cpu,monitoring/0003ad6d/hostname,monitoring/0003ad6d/memory,monitoring/0003ad6d/pid,monitoring/0003ad6d/stderr,monitoring/0003ad6d/stdout,monitoring/0003ad6d/tid,monitoring/00a5a5ef/hostname,monitoring/00a5a5ef/pid,monitoring/00a5a5ef/tid,monitoring/010a5ec3/cpu,monitoring/010a5ec3/gpu,monitoring/010a5ec3/gpu_memory,monitoring/010a5ec3/hostname,monitoring/010a5ec3/memory,monitoring/010a5ec3/pid,monitoring/010a5ec3/stderr,monitoring/010a5ec3/stdout,monitoring/010a5ec3/tid,monitoring/01226e44/hostname,monitoring/01226e44/pid,monitoring/01226e44/tid,monitoring/0122af92/cpu,monitoring/0122af92/gpu,monitoring/0122af92/gpu_memory,monitoring/0122af92/hostname,monitoring/0122af92/memory,...,monitoring/fcb20f96/pid,monitoring/fcb20f96/stderr,monitoring/fcb20f96/stdout,monitoring/fcb20f96/tid,monitoring/fcf307ae/cpu,monitoring/fcf307ae/gpu,monitoring/fcf307ae/gpu_memory,monitoring/fcf307ae/hostname,monitoring/fcf307ae/memory,monitoring/fcf307ae/pid,monitoring/fcf307ae/stderr,monitoring/fcf307ae/stdout,monitoring/fcf307ae/tid,monitoring/fd69b944/cpu,monitoring/fd69b944/hostname,monitoring/fd69b944/memory,monitoring/fd69b944/pid,monitoring/fd69b944/stderr,monitoring/fd69b944/stdout,monitoring/fd69b944/tid,monitoring/fdb51be2/hostname,monitoring/fdb51be2/pid,monitoring/fdb51be2/tid,monitoring/fe8f7afb/hostname,monitoring/fe8f7afb/pid,monitoring/fe8f7afb/tid,monitoring/ff7e9d8d/cpu,monitoring/ff7e9d8d/gpu,monitoring/ff7e9d8d/gpu_memory,monitoring/ff7e9d8d/hostname,monitoring/ff7e9d8d/memory,monitoring/ff7e9d8d/pid,monitoring/ff7e9d8d/stderr,monitoring/ff7e9d8d/stdout,monitoring/ff7e9d8d/tid,monitoring/ff988fbc/cpu,monitoring/ff988fbc/gpu,monitoring/ff988fbc/gpu_memory,monitoring/ff988fbc/hostname,monitoring/ff988fbc/memory,monitoring/ff988fbc/pid,monitoring/ff988fbc/stderr,monitoring/ff988fbc/stdout,monitoring/ff988fbc/tid,monitoring/ffb1b36e/hostname,monitoring/ffb1b36e/pid,monitoring/ffb1b36e/tid,monitoring/ffe423d7/hostname,monitoring/ffe423d7/pid,monitoring/ffe423d7/tid
0,2024-09-13 20:12:19.509,,False,,Apogee,LS2023-1344,2024-09-24 02:33:48.696,542,Untitled,commander.pho,2024-09-24 02:53:25.675,1598.984,57447816.0,Inactive,"kdiba,gor01,2006-6-08_14-26-15,one",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-08_14-26-15_sess,2006-6-08_14-26-15,NaN,3d161945bc11ef636a656d365b54598de9a7f627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-10 20:49:04.507,,False,,Apogee,LS2023-1343,2024-09-24 02:33:48.148,399,Untitled,commander.pho,2024-09-24 02:53:25.286,1546.133,39975659.0,Inactive,"kdiba,two,gor01,2006-6-07_16-40-19",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-07_16-40-19_sess,2006-6-07_16-40-19,figures_kdiba_gor01_two_2006-6-07_16-40-19.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-10 20:26:31.650,,False,,Apogee,LS2023-1342,2024-09-24 02:33:47.421,356,Untitled,commander.pho,2024-09-24 02:53:23.329,1488.411,28291307.0,Inactive,"2006-6-07_11-26-53,one,kdiba,gor01",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-07_11-26-53_sess,2006-6-07_11-26-53,figures_kdiba_gor01_one_2006-6-07_11-26-53.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
runs_table_df.to_csv('output/neptune_runs_csv.csv')

In [3]:
# Filter rows based on column: 'sys/creation_time'
from datetime import datetime

active_runs_table_df: pd.DataFrame = deepcopy(runs_table_df)

# Sort by column: 'sys/creation_time' (ascending)
active_runs_table_df = active_runs_table_df.sort_values(['sys/creation_time'])
active_runs_table_df = active_runs_table_df[active_runs_table_df['sys/creation_time'] > datetime.strptime('2024-08-01T00:30:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
# active_runs_table_df = active_runs_table_df[active_runs_table_df['sys/creation_time'] > datetime.strptime('2024-09-01T00:30:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
# active_runs_table_df = active_runs_table_df[active_runs_table_df['sys/ping_time'] > datetime.strptime('2024-09-01T00:30:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
active_runs_table_df
# Grouped on column: 'session_descriptor_string'
# active_runs_table_df = active_runs_table_df.groupby(['session_descriptor_string']).count().reset_index()[['session_descriptor_string']]
# Performed 1 aggregation grouped on column: 'session_descriptor_string'
most_recent_runs_table_df = active_runs_table_df.groupby(['session_descriptor_string']).agg(sysid_last=('sys/id', 'last')).reset_index()
most_recent_runs_table_df

active_runs_table_df = active_runs_table_df[np.isin(active_runs_table_df['sys/id'], most_recent_runs_table_df['sysid_last'])] # find only the rows that match the latest row_id
active_runs_table_df

,sys/creation_time,sys/description,sys/failed,sys/group_tags,sys/hostname,sys/id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,sys/state,sys/tags,sys/trashed,animal,exper_name,format_name,session_descriptor_string,session_name,source_code/entrypoint,source_code/git,monitoring/0003ad6d/cpu,monitoring/0003ad6d/hostname,monitoring/0003ad6d/memory,monitoring/0003ad6d/pid,monitoring/0003ad6d/stderr,monitoring/0003ad6d/stdout,monitoring/0003ad6d/tid,monitoring/00a5a5ef/hostname,monitoring/00a5a5ef/pid,monitoring/00a5a5ef/tid,monitoring/010a5ec3/cpu,monitoring/010a5ec3/gpu,monitoring/010a5ec3/gpu_memory,monitoring/010a5ec3/hostname,monitoring/010a5ec3/memory,monitoring/010a5ec3/pid,monitoring/010a5ec3/stderr,monitoring/010a5ec3/stdout,monitoring/010a5ec3/tid,monitoring/01226e44/hostname,monitoring/01226e44/pid,monitoring/01226e44/tid,monitoring/0122af92/cpu,monitoring/0122af92/gpu,monitoring/0122af92/gpu_memory,monitoring/0122af92/hostname,monitoring/0122af92/memory,...,monitoring/fcb20f96/pid,monitoring/fcb20f96/stderr,monitoring/fcb20f96/stdout,monitoring/fcb20f96/tid,monitoring/fcf307ae/cpu,monitoring/fcf307ae/gpu,monitoring/fcf307ae/gpu_memory,monitoring/fcf307ae/hostname,monitoring/fcf307ae/memory,monitoring/fcf307ae/pid,monitoring/fcf307ae/stderr,monitoring/fcf307ae/stdout,monitoring/fcf307ae/tid,monitoring/fd69b944/cpu,monitoring/fd69b944/hostname,monitoring/fd69b944/memory,monitoring/fd69b944/pid,monitoring/fd69b944/stderr,monitoring/fd69b944/stdout,monitoring/fd69b944/tid,monitoring/fdb51be2/hostname,monitoring/fdb51be2/pid,monitoring/fdb51be2/tid,monitoring/fe8f7afb/hostname,monitoring/fe8f7afb/pid,monitoring/fe8f7afb/tid,monitoring/ff7e9d8d/cpu,monitoring/ff7e9d8d/gpu,monitoring/ff7e9d8d/gpu_memory,monitoring/ff7e9d8d/hostname,monitoring/ff7e9d8d/memory,monitoring/ff7e9d8d/pid,monitoring/ff7e9d8d/stderr,monitoring/ff7e9d8d/stdout,monitoring/ff7e9d8d/tid,monitoring/ff988fbc/cpu,monitoring/ff988fbc/gpu,monitoring/ff988fbc/gpu_memory,monitoring/ff988fbc/hostname,monitoring/ff988fbc/memory,monitoring/ff988fbc/pid,monitoring/ff988fbc/stderr,monitoring/ff988fbc/stdout,monitoring/ff988fbc/tid,monitoring/ffb1b36e/hostname,monitoring/ffb1b36e/pid,monitoring/ffb1b36e/tid,monitoring/ffe423d7/hostname,monitoring/ffe423d7/pid,monitoring/ffe423d7/tid
18,2024-08-29 15:37:46.293,,False,,gl3126.arc-ts.umich.edu,LS2023-1326,2024-09-24 02:36:17.400,270,Untitled,commander.pho,2024-09-24 02:53:25.448,1296.911,18434390.0,Inactive,"kdiba,2006-6-12_15-55-31,gor01,one",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-12_15-55-31_sess,2006-6-12_15-55-31,figures_kdiba_gor01_one_2006-6-12_15-55-31.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2024-08-29 15:40:40.223,,False,,gl3126.arc-ts.umich.edu,LS2023-1327,2024-08-29 15:52:50.778,731,Untitled,commander.pho,2024-08-29 15:52:50.778,654.847,41033626.0,Inactive,"two,gor01,2006-6-07_16-40-19,kdiba",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-07_16-40-19_sess,2006-6-07_16-40-19,figures_kdiba_gor01_two_2006-6-07_16-40-19.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2024-08-29 15:54:39.196,,False,,gl3126.arc-ts.umich.edu,LS2023-1328,2024-09-24 02:36:17.899,568,Untitled,commander.pho,2024-09-24 02:53:25.944,1573.366,50545354.0,Inactive,"kdiba,gor01,2006-6-08_21-16-25,two",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-08_21-16-25_sess,2006-6-08_21-16-25,figures_kdiba_gor01_two_2006-6-08_21-16-25.py,NaN,NaN,NaN,NaN,

,session_descriptor_string,sysid_last
0,kdiba_gor01_one_2006-6-07_11-26-53_sess,LS2023-1342
1,kdiba_gor01_one_2006-6-08_14-26-15_sess,LS2023-1344
2,kdiba_gor01_one_2006-6-09_1-22-43_sess,LS2023-1339
3,kdiba_gor01_one_2006-6-12_15-55-31_sess,LS2023-1326
4,kdiba_gor01_two_2006-6-07_16-40-19_sess,LS2023-1343
5,kdiba_gor01_two_2006-6-08_21-16-25_sess,LS2023-1328
6,kdiba_gor01_two_2006-6-09_22-24-40_sess,LS2023-1329
7,kdiba_gor01_two_2006-6-12_16-53-46_sess,LS2023-1330
8,kdiba_pin01_one_11-02_17-46-44_sess,LS2023-1335
9,kdiba_pin01_one_11-02_19-28-0_sess,LS2023-1336


,sys/creation_time,sys/description,sys/failed,sys/group_tags,sys/hostname,sys/id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,sys/state,sys/tags,sys/trashed,animal,exper_name,format_name,session_descriptor_string,session_name,source_code/entrypoint,source_code/git,monitoring/0003ad6d/cpu,monitoring/0003ad6d/hostname,monitoring/0003ad6d/memory,monitoring/0003ad6d/pid,monitoring/0003ad6d/stderr,monitoring/0003ad6d/stdout,monitoring/0003ad6d/tid,monitoring/00a5a5ef/hostname,monitoring/00a5a5ef/pid,monitoring/00a5a5ef/tid,monitoring/010a5ec3/cpu,monitoring/010a5ec3/gpu,monitoring/010a5ec3/gpu_memory,monitoring/010a5ec3/hostname,monitoring/010a5ec3/memory,monitoring/010a5ec3/pid,monitoring/010a5ec3/stderr,monitoring/010a5ec3/stdout,monitoring/010a5ec3/tid,monitoring/01226e44/hostname,monitoring/01226e44/pid,monitoring/01226e44/tid,monitoring/0122af92/cpu,monitoring/0122af92/gpu,monitoring/0122af92/gpu_memory,monitoring/0122af92/hostname,monitoring/0122af92/memory,...,monitoring/fcb20f96/pid,monitoring/fcb20f96/stderr,monitoring/fcb20f96/stdout,monitoring/fcb20f96/tid,monitoring/fcf307ae/cpu,monitoring/fcf307ae/gpu,monitoring/fcf307ae/gpu_memory,monitoring/fcf307ae/hostname,monitoring/fcf307ae/memory,monitoring/fcf307ae/pid,monitoring/fcf307ae/stderr,monitoring/fcf307ae/stdout,monitoring/fcf307ae/tid,monitoring/fd69b944/cpu,monitoring/fd69b944/hostname,monitoring/fd69b944/memory,monitoring/fd69b944/pid,monitoring/fd69b944/stderr,monitoring/fd69b944/stdout,monitoring/fd69b944/tid,monitoring/fdb51be2/hostname,monitoring/fdb51be2/pid,monitoring/fdb51be2/tid,monitoring/fe8f7afb/hostname,monitoring/fe8f7afb/pid,monitoring/fe8f7afb/tid,monitoring/ff7e9d8d/cpu,monitoring/ff7e9d8d/gpu,monitoring/ff7e9d8d/gpu_memory,monitoring/ff7e9d8d/hostname,monitoring/ff7e9d8d/memory,monitoring/ff7e9d8d/pid,monitoring/ff7e9d8d/stderr,monitoring/ff7e9d8d/stdout,monitoring/ff7e9d8d/tid,monitoring/ff988fbc/cpu,monitoring/ff988fbc/gpu,monitoring/ff988fbc/gpu_memory,monitoring/ff988fbc/hostname,monitoring/ff988fbc/memory,monitoring/ff988fbc/pid,monitoring/ff988fbc/stderr,monitoring/ff988fbc/stdout,monitoring/ff988fbc/tid,monitoring/ffb1b36e/hostname,monitoring/ffb1b36e/pid,monitoring/ffb1b36e/tid,monitoring/ffe423d7/hostname,monitoring/ffe423d7/pid,monitoring/ffe423d7/tid
18,2024-08-29 15:37:46.293,,False,,gl3126.arc-ts.umich.edu,LS2023-1326,2024-09-24 02:36:17.400,270,Untitled,commander.pho,2024-09-24 02:53:25.448,1296.911,18434390.0,Inactive,"kdiba,2006-6-12_15-55-31,gor01,one",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-12_15-55-31_sess,2006-6-12_15-55-31,figures_kdiba_gor01_one_2006-6-12_15-55-31.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2024-08-29 15:54:39.196,,False,,gl3126.arc-ts.umich.edu,LS2023-1328,2024-09-24 02:36:17.899,568,Untitled,commander.pho,2024-09-24 02:53:25.944,1573.366,50545354.0,Inactive,"kdiba,gor01,2006-6-08_21-16-25,two",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-08_21-16-25_sess,2006-6-08_21-16-25,figures_kdiba_gor01_two_2006-6-08_21-16-25.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2024-08-29 16:03:17.406,,False,,gl3126.arc-ts.umich.edu,LS2023-1329,2024-09-24 02:36:18.655,1107,Untitled,commander.pho,2024-09-24 02:53:27.767,2035.770,60868116.0,Inactive,"two,2006-6-09_22-24-40,gor01,kdiba",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-09_22-24-40_sess,2006-6-09_22-24-40,figures_kdiba_gor01_two_2006-6-09_22-24-40.py,NaN,NaN,NaN,Na

In [ ]:
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import AutoValueConvertingNeptuneRun

known_figure_output_keys = ['outputs/figures/display_fn_name:bidir_track_remap/subplot_name:Track Remapping']

## INPUTS: neptuner, active_runs_table_df
# neptune_root_output_path = Path('EXTERNAL/PhoDibaPaper2024Book/data/neptune').resolve()
neptune_root_output_path = Path(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\PhoDibaPaper2024Book\data\neptune").resolve()
neptune_root_output_path.mkdir(exist_ok=True)
neptune_project_output_path = neptune_root_output_path.joinpath(neptuner.project_name).resolve()
neptune_project_output_path.mkdir(exist_ok=True, parents=True)

# Dictionary to hold the paths of figures for each run
figures_paths = {}

# Iterate over each run in the DataFrame
for run_id in active_runs_table_df['sys/id']:
    try:
        # Access the run by ID
        run: neptune.Run = neptune.init_run(with_id=str(run_id), project=neptuner.project_name, api_token=neptuner.api_token, mode="read-only")
        print(f'run_id: {run_id}\n\trun.get_url(): {run.get_url()}')

        ## make the folder for this run
        neptune_run_output_path = neptune_project_output_path.joinpath(str(run_id)).resolve()
        neptune_run_output_path.mkdir(exist_ok=True)
        print(f'\tneptune_run_output_path: {neptune_run_output_path}')
        
        # neptune_run_figures_output_path = neptune_run_output_path.joinpath(f'figures_{run_id}').resolve()
        neptune_run_figures_output_path = neptune_run_output_path.joinpath(f'figures').resolve()
        neptune_run_figures_output_path.mkdir(exist_ok=True)
        print(f'\tneptune_run_figures_output_path: {neptune_run_figures_output_path}')
        

        for a_full_key_path in known_figure_output_keys:
            run[a_full_key_path].download(destination=neptune_run_figures_output_path)
            
        # Fetch 'outputs/figures' and download the folder or individual files
        # Access the namespace for 'outputs/figures'
        figures_namespace = run['outputs/figures']

        # # Fetch the list of files under this namespace
        # files_list = figures_namespace.fetch_files()
        # print(f'\tfiles_list: {files_list}')
        
        # # run['outputs/figures'].download(destination=neptune_run_figures_output_path)
        # # run["artifacts/images"].download(destination=neptune_run_figures_output_path)
        # # Store the download location for reference

        figures_paths[run_id] = neptune_run_figures_output_path

    except Exception as e:
        print(f"Failed to fetch figures for run {run_id}: {e}")
        
## OUTPUTS: active_runs_table_df
figures_paths

In [ ]:
figures_namespace.keys()

In [ ]:
# Fetch all experiments
experiments = neptuner.project.fetch_experiments_table().to_pandas()
experiments

In [ ]:
# Initialize the neptune client
# neptune.init(project=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])

# project = neptune.get_project()
# project

In [ ]:


if neptuner.run is None:
    neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
    params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
    neptuner.run["parameters"] = params
    neptuner.outputs = neptuner.run['outputs']
    neptuner.figures = neptuner.outputs['figures']

neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run

# run = neptune.init_run(source_files=["**/*.dvc"])

# # Pre-execution dataframe view:
# run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"



known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs